Preparing data for the analysis of accuraciers.

*1) Excluding outliers per participant per condition, based on the outliers defined for the analysis of response times*\
*2) Merging the outliers-free data for the correctly answered word trials, with the newly created outliers-free file for the incorrectly answered word trials.*

In [1]:
import pandas as pd

# Importing the files created with the previous Jupyter Notebook code.
df_all_data_with_boundaries = pd.read_csv("all_data_with_boundaries.csv")
df_word_correct_without_outliers = pd.read_csv("file_with_data_without_outliers_response_times.csv")

df_all_people_without_outliers_analysis_of_accuracies = pd.DataFrame()

# The file with correct word trials without outliers was prepared with the previous Jupyter Notebook code.
# Here, outliers for the incorrect word trials are excluded based on the lower and upper boundaries calculated for the response time analysis with correct word trials.
for subject_nr, df in df_all_data_with_boundaries.groupby("subject_nr"):

    df_word_incorrect = df[
        (df["correct_keyboard_response_1"] == 0) & (df["target_type"] == "word")
    ]
    df_subject_without_outliers = pd.DataFrame()

    for flanker_condition, df_condition in df_word_incorrect.groupby(
        "flanker_condition"
    ):

        df_cond_without_outliers = df_condition[
            (
                (df_condition["response_time_keyboard_response_1"])
                > (df_condition["lower_boundary"])
            )
            & (
                (df_condition["response_time_keyboard_response_1"])
                < (df_condition["upper_boundary"])
            )
        ]
        # This dataframe containes outliers-free incorreclty answered trials per participant.
        df_subject_without_outliers = pd.concat(
            [df_cond_without_outliers, df_subject_without_outliers], ignore_index=True
        )
    # This dataframe contains outliers-free incorreclty answered trials for all of the participants.
    df_all_people_without_outliers_analysis_of_accuracies = pd.concat(
        [
            df_all_people_without_outliers_analysis_of_accuracies,
            df_subject_without_outliers,
        ],
        ignore_index=True,
    )

df_all_people_without_outliers_analysis_of_accuracies_dropped = (
    df_all_people_without_outliers_analysis_of_accuracies.drop(
        columns=["upper_boundary", "lower_boundary"]
    )
)
# Merging the file with the correct word trials without outliers and the file with the incorrect word trials without outliers for the analysis of accuracies.
df_all = pd.concat(
    [
        df_all_people_without_outliers_analysis_of_accuracies_dropped,
        df_word_correct_without_outliers,
    ],
    ignore_index=True,
)

df_all.to_csv("file_with_data_without_outliers_accuracies.csv")